In [16]:
!pip install pandas numpy fuzzywuzzy python-Levenshtein

In [17]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

# Load the Dataset

In [18]:
#Load CSV file into pandas

try:
    df = pd.read_csv('dataset.csv')
    print("Dataset loaded successfully!")
    print(f"Initial shape: {df.shape}")
except FileNotFoundError:
    print("Please provide the correct path to your CSV file")

Dataset loaded successfully!
Initial shape: (27940, 26)


# Standardization of Job Titles using Fuzzy Matching

In [19]:
"""
Convert to lowercase and remove extra whitespace.
Remove special characters except basic punctuation
"""
def clean_text(text):
    if pd.isna(text):
        return text
    text = str(text).lower().strip()
    text = re.sub(r'[^\w\s.,-]', '', text)
    return text

In [20]:
"""
Standardize job titles using fuzzy matching
threshold: similarity score threshold (0-100)
"""
def standardize_job_titles(df, column_name, threshold=50):
    # Clean the job titles first
    df[column_name] = df[column_name].apply(clean_text)
    print(f"Starting standardization of '{column_name}' column...")
    
    # Get unique job titles
    unique_titles = df[column_name].dropna().unique()
    print(f"Found {len(unique_titles)} unique job titles to process")
    
    # Create clusters of similar titles
    standardized_titles = {}
    remaining_titles = list(unique_titles)
    total_clusters = 0
    
    while remaining_titles:
        base_title = remaining_titles.pop(0)
        print(f"\nProcessing base title: '{base_title}'")
        matches = process.extract(base_title, remaining_titles, 
                               scorer=fuzz.token_sort_ratio)
        
        # Find similar titles above threshold
        similar_titles = [m[0] for m in matches if m[1] >= threshold]
        
        if similar_titles:
            total_clusters += 1
            print(f"Found {len(similar_titles)} similar titles (threshold={threshold}):")
            for match, score in [(m[0], m[1]) for m in matches if m[1] >= threshold]:
                print(f"  - '{match}' (similarity score: {score}) -> standardized to '{base_title}'")
            standardized_titles[base_title] = similar_titles
            # Remove matched titles from remaining list
            remaining_titles = [t for t in remaining_titles if t not in similar_titles]
            print(f"Remaining titles to process: {len(remaining_titles)}")
        else:
            print("  No matches found above threshold")
    
    # Create mapping dictionary
    title_mapping = {}
    for standard_title, similar_list in standardized_titles.items():
        title_mapping[standard_title] = standard_title
        for similar in similar_list:
            title_mapping[similar] = standard_title
    
    # Apply mapping to dataframe
    print(f"\nApplying {len(title_mapping)} mappings to the dataset...")
    df[f'standardized_{column_name}'] = df[column_name].map(
        lambda x: title_mapping.get(x, x) if pd.notna(x) else x
    )
    
    # Summary
    print(f"Standardization complete!")
    print(f"Total clusters formed: {total_clusters}")
    print(f"Unique titles reduced from {len(unique_titles)} to {df[f'standardized_{column_name}'].nunique()}")
    
    return df, title_mapping

In [21]:
"""
Apply Standardization
"""
def apply_standardization(df):
    job_column = 'Job title'
    
    # Initial data exploration
    print("Initial unique job titles:", df[job_column].nunique())
    print("Missing values:", df[job_column].isna().sum())
    
    # Apply fuzzy matching for remaining titles
    df, title_mapping = standardize_job_titles(df, f'{job_column}')
    
    return df

try:
    # Process the data
    processed_df = apply_standardization(df)
    
    # Display sample of results
    print("\nSample of processed data:")
    print(processed_df[[
        'Job title',
        'standardized_Job title'
    ]].head(10))

except Exception as e:
    print(f"An error occurred: {str(e)}")

Initial unique job titles: 14288
Missing values: 0
Starting standardization of 'Job title' column...
Found 12062 unique job titles to process

Processing base title: 'research and instruction librarian'
Found 5 similar titles (threshold=50):
  - 'research  instruction librarian' (similarity score: 94) -> standardized to 'research and instruction librarian'
  - 'reference and instruction librarian' (similarity score: 90) -> standardized to 'research and instruction librarian'
  - 'head of research and instruction' (similarity score: 76) -> standardized to 'research and instruction librarian'
  - 'instruction librarian' (similarity score: 76) -> standardized to 'research and instruction librarian'
  - 'professor  instruction and assessment librarian' (similarity score: 75) -> standardized to 'research and instruction librarian'
Remaining titles to process: 12056

Processing base title: 'change  internal communications manager'
Found 5 similar titles (threshold=50):
  - 'internal communic

Found 5 similar titles (threshold=50):
  - 'senior it manager' (similarity score: 90) -> standardized to 'senior manager'
  - 'senior manager, it' (similarity score: 90) -> standardized to 'senior manager'
  - 'senior manager, hr' (similarity score: 90) -> standardized to 'senior manager'
  - 'senior hr manager' (similarity score: 90) -> standardized to 'senior manager'
  - 'hr senior manager' (similarity score: 90) -> standardized to 'senior manager'
Remaining titles to process: 11948

Processing base title: 'assistant director of academic advising'
Found 5 similar titles (threshold=50):
  - 'assistant director, academic  advising' (similarity score: 96) -> standardized to 'assistant director of academic advising'
  - 'director of academic advising' (similarity score: 85) -> standardized to 'assistant director of academic advising'
  - 'assistant director of admission' (similarity score: 83) -> standardized to 'assistant director of academic advising'
  - 'assistant director of admiss

Found 5 similar titles (threshold=50):
  - 'engineering lead' (similarity score: 90) -> standardized to 'lead engineer'
  - 'lead qa engineer' (similarity score: 90) -> standardized to 'lead engineer'
  - 'lead web engineer' (similarity score: 87) -> standardized to 'lead engineer'
  - 'lead ios engineer' (similarity score: 87) -> standardized to 'lead engineer'
  - 'lead data engineer' (similarity score: 84) -> standardized to 'lead engineer'
Remaining titles to process: 11828

Processing base title: 'senior copywriter'
Found 5 similar titles (threshold=50):
  - 'junior copywriter' (similarity score: 88) -> standardized to 'senior copywriter'
  - 'editor, copywriter' (similarity score: 88) -> standardized to 'senior copywriter'
  - 'sr copywriter' (similarity score: 87) -> standardized to 'senior copywriter'
  - 'senior copyeditor' (similarity score: 82) -> standardized to 'senior copywriter'
  - 'senior recruiter' (similarity score: 79) -> standardized to 'senior copywriter'
Remainin

Found 5 similar titles (threshold=50):
  - 'hr coordinator' (similarity score: 88) -> standardized to 'coordinator'
  - 'coordinator ii' (similarity score: 88) -> standardized to 'coordinator'
  - 'qa coordinator' (similarity score: 88) -> standardized to 'coordinator'
  - 'rn coordinator' (similarity score: 88) -> standardized to 'coordinator'
  - 'sr. coordinator' (similarity score: 88) -> standardized to 'coordinator'
Remaining titles to process: 11714

Processing base title: 'senior advisor'
Found 5 similar titles (threshold=50):
  - 'senior it advisor' (similarity score: 90) -> standardized to 'senior advisor'
  - 'senior hr advisor' (similarity score: 90) -> standardized to 'senior advisor'
  - 'senior aditor' (similarity score: 89) -> standardized to 'senior advisor'
  - 'senior auditor' (similarity score: 86) -> standardized to 'senior advisor'
  - 'senior donor advisor' (similarity score: 82) -> standardized to 'senior advisor'
Remaining titles to process: 11708

Processing ba

Found 5 similar titles (threshold=50):
  - 'technical writer' (similarity score: 81) -> standardized to 'tech writer'
  - 'tv writer' (similarity score: 80) -> standardized to 'tech writer'
  - 'hr writer' (similarity score: 80) -> standardized to 'tech writer'
  - 'speechwriter' (similarity score: 78) -> standardized to 'tech writer'
  - 'techical writer ii' (similarity score: 76) -> standardized to 'tech writer'
Remaining titles to process: 11630

Processing base title: 'supervisory archivist'
Found 5 similar titles (threshold=50):
  - 'archives supervisor' (similarity score: 90) -> standardized to 'supervisory archivist'
  - 'senior archivist' (similarity score: 81) -> standardized to 'supervisory archivist'
  - 'it supervisor' (similarity score: 76) -> standardized to 'supervisory archivist'
  - 'shift supervisor' (similarity score: 76) -> standardized to 'supervisory archivist'
  - 'supervisor hr specialist' (similarity score: 76) -> standardized to 'supervisory archivist'
Remaini

...

Found 5 similar titles (threshold=50):
  - 'high ropes instructor' (similarity score: 68) -> standardized to 'workforce instructor'
  - 'sr analyst - workforce technology' (similarity score: 63) -> standardized to 'workforce instructor'
  - 'instructor, psychology' (similarity score: 63) -> standardized to 'workforce instructor'
  - 'math instructor tenured' (similarity score: 60) -> standardized to 'workforce instructor'
  - 'workforce optimisation analyst' (similarity score: 60) -> standardized to 'workforce instructor'
Remaining titles to process: 1779

Processing base title: 'family education'
Found 5 similar titles (threshold=50):
  - 'educational technology manager' (similarity score: 57) -> standardized to 'family education'
  - 'catering sales' (similarity score: 53) -> standardized to 'family education'
  - 'higher education assistant' (similarity score: 52) -> standardized to 'family education'
  - 'pipeline operations' (similarity score: 51) -> standardized to 'family educat

Found 5 similar titles (threshold=50):
  - 'telephonic practice area consultant' (similarity score: 57) -> standardized to 'apprentice mechanic for construction machinery'
  - 'technical documentation manager' (similarity score: 55) -> standardized to 'apprentice mechanic for construction machinery'
  - 'distribution partner account manager' (similarity score: 54) -> standardized to 'apprentice mechanic for construction machinery'
  - 'apprentice early years assistant' (similarity score: 54) -> standardized to 'apprentice mechanic for construction machinery'
  - 'teaching instructor of first year writing' (similarity score: 53) -> standardized to 'apprentice mechanic for construction machinery'
Remaining titles to process: 1557

Processing base title: 'officemarketing manager'
Found 5 similar titles (threshold=50):
  - 'trucking back office manager' (similarity score: 75) -> standardized to 'officemarketing manager'
  - 'group marketing manager' (similarity score: 74) -> standardized t

Found 5 similar titles (threshold=50):
  - 'hr recruiter' (similarity score: 53) -> standardized to 'minister united church'
  - 'it intern' (similarity score: 52) -> standardized to 'minister united church'
  - 'threat hunter' (similarity score: 51) -> standardized to 'minister united church'
  - 'commercial real estate underwriter' (similarity score: 50) -> standardized to 'minister united church'
  - 'hvac installer' (similarity score: 50) -> standardized to 'minister united church'
Remaining titles to process: 1301

Processing base title: 'ghostwriter'
Found 5 similar titles (threshold=50):
  - 'scriptwriter' (similarity score: 70) -> standardized to 'ghostwriter'
  - 'copywritier' (similarity score: 64) -> standardized to 'ghostwriter'
  - 'subtitler' (similarity score: 60) -> standardized to 'ghostwriter'
  - 'strategic writer' (similarity score: 59) -> standardized to 'ghostwriter'
  - 'scientific writer' (similarity score: 57) -> standardized to 'ghostwriter'
Remaining titles t

Found 5 similar titles (threshold=50):
  - 'general ledger accountant' (similarity score: 65) -> standardized to 'payroll accountant'
  - 'account lead' (similarity score: 60) -> standardized to 'payroll accountant'
  - 'financial reporting accountant' (similarity score: 58) -> standardized to 'payroll accountant'
  - 'payroll and benefit coordinator' (similarity score: 57) -> standardized to 'payroll accountant'
  - 'audit  accounts assistant' (similarity score: 57) -> standardized to 'payroll accountant'
Remaining titles to process: 1001

Processing base title: 'senior technical writer contract'
Found 5 similar titles (threshold=50):
  - 'senior technical product owner' (similarity score: 71) -> standardized to 'senior technical writer contract'
  - 'security technical consultant' (similarity score: 66) -> standardized to 'senior technical writer contract'
  - 'technical consultant data scienceai' (similarity score: 63) -> standardized to 'senior technical writer contract'
  - 'senio

Found 5 similar titles (threshold=50):
  - 'senior ios developerconsultant' (similarity score: 60) -> standardized to 'senior human rights advisor'
  - 'advisor, disease and vector control' (similarity score: 59) -> standardized to 'senior human rights advisor'
  - 'senior human resources executive' (similarity score: 58) -> standardized to 'senior human rights advisor'
  - 'senior draughtsperson' (similarity score: 58) -> standardized to 'senior human rights advisor'
  - 'senior inbound marketer' (similarity score: 56) -> standardized to 'senior human rights advisor'
Remaining titles to process: 553

Processing base title: 'event monitor'
Found 4 similar titles (threshold=50):
  - 'senior geochemist' (similarity score: 53) -> standardized to 'event monitor'
  - 'monitoring  evaluation manager' (similarity score: 52) -> standardized to 'event monitor'
  - 'development officer monthly giving' (similarity score: 51) -> standardized to 'event monitor'
  - 'telecommunicator iv' (similarity

# Analyze Standardization Quality

In [22]:
"""
Additional analysis of standardization quality
"""
def analyze_standardization_quality(df):
    # Count occurrences of each standardized title
    title_counts = df['standardized_Job title'].value_counts()
    print("\nTop 10 standardized job titles:")
    print(title_counts.head(10))
    
    # Check for remaining inconsistencies
    potential_issues = title_counts[title_counts < 5].index
    print(f"\nNumber of titles appearing less than 5 times: {len(potential_issues)}")
    
    return title_counts

# Run quality analysis
title_counts = analyze_standardization_quality(processed_df)


Top 10 standardized job titles:
software engineer           415
project manager             321
director                    280
senior software engineer    254
program manager             218
executive assistant         213
manager                     200
teacher                     186
librarian                   175
product manager             161
Name: standardized_Job title, dtype: int64

Number of titles appearing less than 5 times: 162


#  Feature Construction for Adjusted Salary

In [23]:
"""
Define approximate conversion rates to USD as of April 2025 (update these as needed)
"""
conversion_rates = {
    'USD': 1.0,
    'GBP': 1.32,    # British Pound
    'CAD': 0.74,    # Canadian Dollar
    'EUR': 1.10,    # Euro
    'AUD': 0.67,    # Australian Dollar
    'SGD': 0.76,    # Singapore Dollar
    'PLN': 0.25,    # Polish Zloty
    'INR': 0.012,   # Indian Rupee
    'TRY': 0.029,   # Turkish Lira
    'NGN': 0.00061, # Nigerian Naira
    'COP': 0.00024, # Colombian Peso
    'INR': 0.012,    # Indian Rupee
    'DKK': 0.15     # Danish Kronor
}

"""
Function to clean and convert salary/bonus to float
"""
def clean_numeric(value):
    if pd.isna(value) or value == '':
        return 0.0
    try:
        # Remove commas and convert to float
        return float(str(value).replace(',', ''))
    except ValueError:
        return 0.0
    
"""
Clean the salary and bonus columns
"""
processed_df['Annual salary'] = processed_df['Annual salary'].apply(clean_numeric)
processed_df['Bonuses'].fillna(0, inplace=True)
processed_df['Bonuses'] = processed_df['Bonuses'].apply(clean_numeric)


"""
Determine currency and convert to USD
"""
def convert_to_usd(row):
    currency = row['Currency']
    salary = row['Annual salary']
    bonus = row['Bonuses']
    
    # If currency is "Other", try to get it from the "Other Currency" field
    if currency == 'Other':
        other_currency = str(row['Other Currency']).strip()
        # Map common "Other" currencies to our conversion rates
        currency_map = {
            'Singapore Dollara': 'SGD',
            'Singaporw': 'SGD',
            'PLN': 'PLN',
            'INR': 'INR',
            'TRY': 'TRY',
            'NGN': 'NGN',
            'COP': 'COP',
            'Rupees': 'Rupees'
        }
        currency = currency_map.get(other_currency, 'USD')
    
    # Get conversion rate (default to 1.0 if currency not found)
    rate = conversion_rates.get(currency, 1.0)
    
    # Convert total compensation (salary + bonus) to USD
    total_usd = (salary + bonus) * rate
    return total_usd

In [24]:
#Create the new "Salary in USD" column
processed_df['Salary in USD'] = processed_df.apply(convert_to_usd, axis=1)


# Apply Purchasing Power Parity Adjustment
ppp_factors = {'USD': 1.0,
               'GBP': 0.73,
               'CAD': 0.75,
               'EUR': 0.8,
               'AUD/NZD': 0.78,
               'CHF': 1.1,
               'ZAR': 0.25,
               'SEK': 0.84,
               'HKD': 0.85,
               'JPY': 0.62}

processed_df['Salary in USD (PPP Adjusted)'] = processed_df.apply(lambda row: row['Salary in USD'] / ppp_factors.get(row['Currency'], 0.50), axis=1)
print(processed_df[['Salary in USD', 'What country do you work in?', 'Salary in USD (PPP Adjusted)']])

       Salary in USD What country do you work in?  \
0            55000.0                United States   
1            77352.0               United Kingdom   
2            34000.0                           US   
3            65000.0                          USA   
4            67000.0                           US   
...              ...                          ...   
27935        25000.0                     Colombia   
27936        55000.0               United States    
27937        94000.0               United States    
27938        64000.0                      denmark   
27939       150000.0                           US   

       Salary in USD (PPP Adjusted)  
0                      55000.000000  
1                     105961.643836  
2                      34000.000000  
3                      65000.000000  
4                      67000.000000  
...                             ...  
27935                  25000.000000  
27936                  55000.000000  
27937                

# Outlier Handling

In [25]:
"""
Apply an upper bound to the adjusted salary values at the 95th percentile
"""

Q1 = processed_df['Salary in USD (PPP Adjusted)'].quantile(0.05)
Q3 = processed_df['Salary in USD (PPP Adjusted)'].quantile(0.95)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
processed_df['Salary in USD (Capped)'] = processed_df['Salary in USD (PPP Adjusted)'].clip(upper=upper_bound)

max_salary_usd = processed_df['Salary in USD (PPP Adjusted)'].max()
max_salary_usd_capped = processed_df['Salary in USD (Capped)'].max()

print(f"Maximum Uncapped Salary in USD': {max_salary_usd}")
print(f"Maximum Capped Salary in USD': {max_salary_usd_capped}")

Maximum Uncapped Salary in USD': 1980000000.0
Maximum Capped Salary in USD': 517900.0


# Feature Aggregation for Location related Features

In [26]:
"""
Standardize country names
"""
def standardize_country(country):
    # Dictionary mapping variations of United States to 'USA'
    us_variations = {
        'USA': 'USA',
        'US': 'USA',
        'United States': 'USA',
        'United Status': 'USA',
        'America': 'USA',
        'U.S.': 'USA',
        'U.S.A.': 'USA',
        'Usa': 'USA',
        'us': 'USA',
        'united states': 'USA'
    }
    # Return 'USA' if the country matches a variation, otherwise return the original country
    return us_variations.get(str(country).strip(), country)

"""
Aggregates location data into a single feature.
"""
def combine_location(row, country_col, state_col, city_col, us_country_value='USA'):
    country = row[country_col]
    state = row[state_col]
    city = row[city_col]
    
    # For US locations, include state if available
    if country == us_country_value and pd.notna(state) and pd.notna(city):
        return f"{country}-{state}-{city}"
    # For non-US locations or missing state/city, omit state
    elif pd.notna(city):
        return f"{country}-{city}"
    # Fallback if city is missing
    return country

In [27]:
"""
Apply Feature Aggregation
"""

# Standardize the country names
processed_df['What country do you work in?'] = processed_df['What country do you work in?'].apply(standardize_country)

# Apply the function to create the new 'Location' column
processed_df['Location'] = processed_df.apply(
    lambda row: combine_location(
        row,
        country_col='What country do you work in?',
        state_col="If you're in the U.S., what state do you work in?",
        city_col='What city do you work in?',
        us_country_value='USA'
    ),
    axis=1
)

# Display the first few rows of the result
print("\nFirst 10 rows with the new 'Location' column:")
print(processed_df[[
    'What country do you work in?',
    "If you're in the U.S., what state do you work in?",
    'What city do you work in?',
    'Location'
]].head(10))


First 10 rows with the new 'Location' column:
  What country do you work in?  \
0                          USA   
1               United Kingdom   
2                          USA   
3                          USA   
4                          USA   
5                          USA   
6                          USA   
7                          USA   
8                          USA   
9                          USA   

  If you're in the U.S., what state do you work in? What city do you work in?  \
0                                     Massachusetts                    Boston   
1                                               NaN                 Cambridge   
2                                         Tennessee               Chattanooga   
3                                         Wisconsin                 Milwaukee   
4                                    South Carolina                Greenville   
5                                     New Hampshire                   Hanover   
6          

In [28]:
# Save the updated dataset to a new CSV file
processed_df.to_csv('processed_dataset.csv', index=False)